In [1]:
import json
from openai import OpenAI
from groq import Groq
import os
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
import pickle
from elasticsearch import Elasticsearch
import pandas as pd
import requests
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from requests.exceptions import HTTPError

import time

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('llm_answer164.pkl', 'rb') as file:
        print(pickle.load(file)[0])

{'answer_llm': 'Tổng mức đầu tư đã được phê duyệt cho 72 dự án có dấu hiệu đầu tư không hiệu quả là khoảng 42.744 tỷ đồng. \n', 'answer_orig': 'Có tổng cộng 72 dự án có dấu hiệu đầu tư không hiệu quả, với tổng mức đầu tư đã được phê duyệt sau cùng khoảng 42.744 tỷ đồng.', 'document': '67dd4b32', 'question': 'Tổng mức đầu tư phê duyệt cho các dự án không hiệu quả là bao nhiêu ?', 'group': 'General'}


In [3]:
with open('llm_answer82.pkl', 'rb') as file:
        print(pickle.load(file)[0])

{'answer_llm': 'Niềm tin tạo lập giá trị mới là triết lý phát triển của IBSC, thể hiện mong muốn trong vòng 3-5 năm tới, Công ty sẽ trở thành một trong những công ty chứng khoán có thương hiệu mạnh trên thị trường và có sự phát triển bền vững, ổn định, đồng đều giữa các lĩnh vực hoạt động. \n', 'answer_orig': 'Tòa nhà văn phòng mới của Apple có tổng kinh phí hơn 5 tỷ USD và được xây dựng với mục tiêu trở thành tòa nhà văn phòng tốt nhất và xanh nhất hành tinh.', 'document': 'c44974f0', 'question': 'Apple có kế hoạch xây dựng một tòa nhà văn phòng hiện đại với mục tiêu gì?', 'group': 'General'}


In [6]:
df_ground_truth = pd.read_csv('../ground_truth_data/ground_truth_data.csv')

ground_truth = df_ground_truth.to_dict(orient='records')
s = ground_truth[1217:1217*2]
s[1200:]

[{'question': 'Tổng mức đầu tư phê duyệt cho các dự án không hiệu quả là bao nhiêu ?',
  'Group': 'General',
  'document': '67dd4b32'},
 {'question': 'Mức đầu tư cho những dự án này được phê duyệt dưới dạng bao nhiêu?',
  'Group': 'General',
  'document': '67dd4b32'},
 {'question': 'Có bao nhiêu dự án bị xác định là đầu tư không hiệu quả?',
  'Group': 'General',
  'document': '67dd4b32'},
 {'question': 'Có bao nhiêu tỉ đồng được phê duyệt cho các dự án có dấu hiệu đầu tư không hiệu quả? ',
  'Group': 'General',
  'document': '67dd4b32'},
 {'question': 'Đức, Ba Lan và Anh đã đứng đầu bảng nào để giành vé vào vòng chung kết World Cup 2018?',
  'Group': 'General',
  'document': 'a2b6ecc0'},
 {'question': 'Ngoài Đức, Ba Lan và Anh, những đội nào còn có cơ hội dự World Cup 2018?',
  'Group': 'General',
  'document': 'a2b6ecc0'},
 {'question': 'Ai là ba đội dẫn đầu bảng E, F và C?',
  'Group': 'General',
  'document': 'a2b6ecc0'},
 {'question': 'Các đội nào có khả năng dự play-off vòng loại 

In [25]:
chunk_size = 15
start_chunk = 0 # Starting chunk index
end_chunk = (1221 // chunk_size)   # Ending chunk index
# print(end_chunk)
for i in range(start_chunk, end_chunk):
    results = []
    chunk_start = i * chunk_size
    chunk_end = chunk_start + chunk_size
    
    print(i + 329, chunk_start, chunk_end)
    # chunk = documents_current[chunk_start:chunk_end]

    # # Use map_progress to process documents
    # processed_results = map_progress(pool, chunk, process_record)
    # results.extend(processed_results)
    # # # Store the results incrementally
    # # for result in processed_results:
    # #     if result is not None:
    # #         doc_id, questions = result
    # #         results[doc_id] = questions

    # # Save the results to a file
    # file_name = f'../data/vietnamese_rag/llm_answer/llm_answer{i + 1}.pkl'
    # with open(file_name, 'wb') as file:
    #     pickle.dump(results, file)

    # # Print out the results
    # print(f"Chunk {i} processed and saved to {file_name}")
    # # print(results)

    # # Wait for 1 minute to reset rate limit
    # time.sleep(20)

329 0 15
330 15 30
331 30 45
332 45 60
333 60 75
334 75 90
335 90 105
336 105 120
337 120 135
338 135 150
339 150 165
340 165 180
341 180 195
342 195 210
343 210 225
344 225 240
345 240 255
346 255 270
347 270 285
348 285 300
349 300 315
350 315 330
351 330 345
352 345 360
353 360 375
354 375 390
355 390 405
356 405 420
357 420 435
358 435 450
359 450 465
360 465 480
361 480 495
362 495 510
363 510 525
364 525 540
365 540 555
366 555 570
367 570 585
368 585 600
369 600 615
370 615 630
371 630 645
372 645 660
373 660 675
374 675 690
375 690 705
376 705 720
377 720 735
378 735 750
379 750 765
380 765 780
381 780 795
382 795 810
383 810 825
384 825 840
385 840 855
386 855 870
387 870 885
388 885 900
389 900 915
390 915 930
391 930 945
392 945 960
393 960 975
394 975 990
395 990 1005
396 1005 1020
397 1020 1035
398 1035 1050
399 1050 1065
400 1065 1080
401 1080 1095
402 1095 1110
403 1110 1125
404 1125 1140
405 1140 1155
406 1155 1170
407 1170 1185
408 1185 1200
409 1200 1215
